# Лабораторная работа #6

## Задание

В городе есть N перекрестков и M дорог (каждая дорога начинается и заканчивается перекрестком, дороги имеют направление). Известно время проезда каждой дороги (время проезда дорог i->j и j->i может быть различным). Определить перекресток для расположения на нем пожарной станции с условием: пожарная машина должна попасть в наиболее удаленный от станции перекресток за минимальное возможное время (пожарная машина может нарушать требования ПДД и ехать по встречному направлению).  Задачу реализовать 2-мя алгоритмами.


In [205]:
from typing import List, Tuple
from collections import namedtuple

In [206]:
class Graph:
    weights: List[List[float]]
    paths: List[List[int]]
    distances: List[List[int]]
        
    def __init__(self, vertices_num):
        self.weights = [[float('inf') for _ in range(vertices_num)] for __ in range(vertices_num)]
        self.paths = [[_ for _ in range(vertices_num)] for __ in range(vertices_num)]
        self.distances = [[1 for _ in range(vertices_num)] for __ in range(vertices_num)]

    def get_num_vertices(self) -> int:
        return len(self.weights)

    def set_weight_between_vertices(self, vertex_1, vertex_2, weight) -> None:
        self.weights[vertex_1][vertex_2] = weight
    
    def get_distance_betwen_vertices(self, vertex_1, vertex_2) -> int:
        index_1, index_2 = vertex_1, vertex_2
        num_steps = 1
        while self.paths[index_1][index_2] != vertex_2:
            index_1 = self.paths[index_1][index_2]
            num_steps += 1
        return num_steps
    
    def set_distances_between_all_vertices_in_graph(self) -> None:
        vertices_num: int = self.get_num_vertices()
            
        for i in range(vertices_num):
            for j in range(vertices_num):
                self.distances[i][j] = self.get_distance_betwen_vertices(i, j)
                
    def find_farthest_vertex_to_the_vertex(self, vertex_index: int) -> int:
        vertices_num: int = self.get_num_vertices()
        max_distance: int = -float('inf')
        farthest_vertex_index: int = vertex_index
        for other_vertex_index in range(vertices_num):
            if other_vertex_index != vertex_index:
                if self.distances[vertex_index][other_vertex_index] > max_distance:
                    max_distance = self.distances[vertex_index][other_vertex_index]
                    farthest_vertex_index = other_vertex_index
                    
        return farthest_vertex_index
    
    def get_weight_between_vertices(self, vertex_index_1: int, vertex_index_2: int) -> float:
        return self.weights[vertex_index_1][vertex_index_2]
                
    def get_neighborhood(self, vertex_index: int):
        return [
            (adjacent_vertex_index, weight) 
                for adjacent_vertex_index, weight in enumerate(self.weights[vertex_index])
                    if weight != float("inf")
        ]


Алгоритм Флоида:

In [207]:
def run_run_floyd(graph: Graph) -> List[List[float]]:
    weights: List[List[float]] = graph.weights
    vertices_num: int = graph.get_num_vertices()
    paths: List[List[int]] = graph.paths

    for iteration in range(vertices_num):
        for i in range(vertices_num):
            for j in range(vertices_num):
                if weights[i][iteration] + weights[iteration][j] < weights[i][j]:
                    weights[i][j] = weights[i][iteration] + weights[iteration][j]
                    paths[i][j] = paths[i][iteration]
                    
    return weights, paths

"пожарная машина может нарушать требования ПДД и ехать по встречному направлению"

In [208]:
def correct_weights(graph: Graph) -> Graph:

    for i, _ in enumerate(graph.weights):
        for j, _ in enumerate(graph.weights):
            if graph.weights[i][j] < graph.weights[j][i]:
                graph.weights[j][i] = graph.weights[i][j]
                graph.distances[j][i] = graph.distances[i][j]
            else:
                graph.weights[i][j] = graph.weights[j][i]
                graph.distances[i][j] = graph.distances[j][i]

    return graph

Для всех вершин  — самую дальнюю, и среди них выбираю с наименьшим временем поездки

In [209]:
def find_firestation_run_floyd(graph: Graph):
    num_of_vertices: int = graph.get_num_vertices()
    farthest_vertex = namedtuple('farthest_vertex', 'current_vertex_index farthest_vertex_index travel_time')
        
    farthest_vertices: List[Tuple[int, int]] = []

    for vertex_index in range(num_of_vertices):
        farthest_vertices.append(
            farthest_vertex(
                vertex_index, 
                graph.find_farthest_vertex_to_the_vertex(vertex_index),
                graph.weights[vertex_index][graph.find_farthest_vertex_to_the_vertex(vertex_index)]
            )
        )
    
    
    return min(farthest_vertices, key=lambda farthest_vertex: farthest_vertex.travel_time)


## Пример

In [216]:
g = Graph(5)
g.set_weight_between_vertices(0, 2, 6)
g.set_weight_between_vertices(0, 4, 3)
g.set_weight_between_vertices(1, 0, 4)
g.set_weight_between_vertices(1, 3, 2)
g.set_weight_between_vertices(1, 4, 1)
g.set_weight_between_vertices(2, 1, 5)
g.set_weight_between_vertices(2, 4, 8)
g.set_weight_between_vertices(3, 1, -1)
g.set_weight_between_vertices(3, 2, 7)
g.set_weight_between_vertices(3, 4, -3)
g.set_weight_between_vertices(4, 2, 8)
g.set_weight_between_vertices(4, 3, 4)

In [217]:
run_run_floyd(g)

([[10, 6, 6, 7, 3],
  [4, 1, 7, 2, -1],
  [9, 5, 12, 7, 4],
  [3, -1, 5, 1, -3],
  [7, 3, 8, 4, 1]],
 [[4, 4, 2, 4, 4],
  [0, 3, 3, 3, 3],
  [1, 1, 1, 1, 1],
  [1, 1, 4, 1, 4],
  [3, 3, 2, 3, 3]])

In [212]:
g.set_distances_between_all_vertices_in_graph()
g.distances

[[4, 3, 1, 2, 1],
 [1, 2, 3, 1, 2],
 [2, 1, 4, 2, 3],
 [2, 1, 2, 2, 1],
 [3, 2, 1, 1, 2]]

In [214]:
correct_weights(g)

In [218]:
find_firestation_run_floyd(g)

farthest_vertex(current_vertex_index=3, farthest_vertex_index=0, travel_time=3)

Aлгоритм Дейкстра

In [103]:
def run_dijkstra(graph: Graph, start: int) -> List[float]:
    distance_from_start_to_other: List[float] = [float('inf') for _ in range(graph.get_num_vertices())]
    distance_from_start_to_other[start] = 0
    
    que: List[Tuple[int, int]] = []
    vertex_distance: Tuple[int, int] = namedtuple('vertex_distance', 'vertex distance_to_vertex')
    que.append(vertex_distance(start, 0))
    
    while que:
        vertex, distance = que.pop()
        
        if distance_from_start_to_other[vertex] < distance:
            continue
               
        for adjacent_vertex, adjacent_distance in graph.get_neighborhood(vertex):
            new_distance: int = distance + adjacent_distance
                
            if new_distance < distance_from_start_to_other[adjacent_vertex]:
                distance_from_start_to_other[adjacent_vertex] = new_distance
                que.append(vertex_distance(adjacent_vertex, new_distance))
        
    return distance_from_start_to_other

In [152]:
def find_firestation_dijkstra(graph: Graph):
    num_of_vertices: int = graph.get_num_vertices()
    farthest_vertex: Tuple[int, int] = namedtuple (
                'farthest_vertex', 'current_vertex_index farthest_vertex_index travel_time'
            )
        
    farthest_vertices: List[Tuple[int, int]] = []

    for vertex_index in range(num_of_vertices):
        farthest_vertices.append(
            farthest_vertex(
                vertex_index, 
                graph.find_farthest_vertex_to_the_vertex(vertex_index),
                run_dijkstra(graph, vertex_index)[graph.find_farthest_vertex_to_the_vertex(vertex_index)]
            )
        )
    
    return min(farthest_vertices, key=lambda farthest_vertex: farthest_vertex.travel_time)

## Пример

In [106]:
graph = Graph(5)
graph.set_weight_between_vertices(0, 2, 6)
graph.set_weight_between_vertices(0, 4, 3)
graph.set_weight_between_vertices(1, 0, 4)
graph.set_weight_between_vertices(1, 3, 2)
graph.set_weight_between_vertices(1, 4, 1)
graph.set_weight_between_vertices(2, 1, 5)
graph.set_weight_between_vertices(2, 4, 8)
graph.set_weight_between_vertices(3, 1, -1)
graph.set_weight_between_vertices(3, 2, 7)
graph.set_weight_between_vertices(3, 4, -3)
graph.set_weight_between_vertices(4, 2, 8)
graph.set_weight_between_vertices(4, 3, 4)

In [153]:
find_firestation_dijkstra(g)

farthest_vertex(current_vertex_index=3, farthest_vertex_index=0, travel_time=3)

## Тесты

In [146]:
import unittest

In [203]:
class TestGraph(unittest.TestCase):
    def setUp(self):
        self.graph_a = Graph(6)
        connectivity_array: List[List[int, int, int]] = [
            [0, 1, 7],
            [0, 5, 14],
            [0, 2, 9],
            [2, 5, 2],
            [2, 3, 11],
            [5, 4, 9],
            [3, 4, 6],
            [1, 3, 15]
        ]   
        for component in connectivity_array:
            self.graph_a.set_weight_between_vertices(component[0], component[1], component[2])
            self.graph_a.set_weight_between_vertices(component[1], component[0], component[2])
        self.farthest_vertex: Tuple[int, int] = namedtuple (
                'farthest_vertex', 'current_vertex_index farthest_vertex_index travel_time'
            )

    def test_run_floyd(self):
        self.assertEqual(
            run_floyd(self.graph_a),
            (
                [
                    [14, 7, 9, 20, 20, 11],
                    [7, 14, 16, 15, 21, 18],
                    [9, 16, 4, 11, 11, 2],
                    [20, 15, 11, 12, 6, 13],
                    [20, 21, 11, 6, 12, 9],
                    [11, 18, 2, 13, 9, 4]
                ],
                [
                    [1, 1, 2, 2, 2, 2],
                    [0, 0, 0, 3, 3, 0],
                    [0, 0, 5, 3, 5, 5],
                    [2, 1, 2, 4, 4, 2],
                    [5, 3, 5, 3, 3, 5],
                    [2, 2, 2, 2, 4, 2]
                ]
            )
        )
            
    def test_run_floyd_negative(self):
        self.assertNotEqual(
            run_floyd(self.graph_a),
            (
                [
                    [11, 7, 9, 20, 20, 11],
                    [7, 14, 16, 15, 21, 18],
                    [9, 16, 4, 11, 11, 2],
                    [20, 15, 11, 12, 6, 13],
                    [20, 21, 11, 6, 12, 9],
                    [11, 18, 2, 13, 9, 4]
                ],
                [
                    [1, 1, 2, 2, 2, 2],
                    [0, 0, 0, 3, 3, 0],
                    [0, 0, 5, 3, 5, 5],
                    [2, 1, 2, 4, 4, 2],
                    [5, 3, 5, 3, 3, 5],
                    [2, 2, 2, 2, 4, 2]
                ]
            )
        )
    
    def test_run_dijkstra(self):
        self.assertEqual(
            run_dijkstra(self.graph_a, 0),
            [0, 7, 9, 20, 20, 11]
        )
        self.assertEqual(
            run_dijkstra(self.graph_a, 1),
            [7, 0, 16, 15, 21, 18]
        )
        self.assertEqual(
            run_dijkstra(self.graph_a, 2),
            [9, 16, 0, 11, 11, 2]
        )
        
    def test_run_dijkstra_negative(self):
        self.assertNotEqual(
            run_dijkstra(self.graph_a, 0),
            [0, 6, 9, 20, 20, 11]
        )
        
        
    def test_find_firestation_floyd(self):
        self.assertEqual(
            find_firestation_floyd(self.graph_a),
            self.farthest_vertex(0, 1, 7)
        )
    
    def test_find_firestation_dijkstra(self):
        self.assertEqual(
            find_firestation_dijkstra(self.graph_a),
            self.farthest_vertex(0, 1, 7)
        )
        

unittest.main(argv=[''], verbosity=2, exit=False)


test_find_firestation_dijkstra (__main__.TestGraph) ... ok
test_find_firestation_floyd (__main__.TestGraph) ... ok
test_run_dijkstra (__main__.TestGraph) ... ok
test_run_dijkstra_negative (__main__.TestGraph) ... ok
test_run_floyd (__main__.TestGraph) ... ok
test_run_floyd_negative (__main__.TestGraph) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.031s

OK
